In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
# Install required packages (only needed the first time)
Pkg.instantiate()

In [ ]:
using Flux
using CUDA
using Plots
using PyCall
using MLDatasets: MNIST
using MLDataPattern
using Flux.Data: DataLoader
using FluxTraining

# "Accept" terms when downloading MNIST image data
ENV["DATADEPS_ALWAYS_ACCEPT"] = true

In [ ]:
# Download CUDA packages (first time only) and show info about their version
CUDA.versioninfo()

# Handwriting recognition

In [ ]:
# Load handwritten digits dataset
x_data = reshape(MNIST.traintensor(Float32), 28, 28, 1, :)
y_data = Flux.onehotbatch(MNIST.trainlabels(), 0:9)

# Split into training, validation, testing
traindata, valdata, testdata = splitobs((x_data, y_data), at=(0.9, 0.05))

# Show sizes
@show size(traindata[1]) size(valdata[1]) size(testdata[1])
@show size(traindata[2]) size(valdata[2]) size(testdata[2]);

In [ ]:
# Plot some images
plots = map(1:25) do i
    p = plot(Gray.(1 .- testdata[1][:, :, 1, i])'; yaxis=nothing, xaxis=nothing)
    label = Flux.onecold(testdata[2][:, i], 0:9)
    title!(p, string(label), titlefontsize=8)
end
plot(plots...)

We will next define the following structure of the neural network:

![](model.png)

The input will be the image, the output will be the 10 probabilities.

In [ ]:
# Build the neural network.
model = Chain(
    Conv((3, 3), 1 => 6, relu),
    AdaptiveMaxPool((13, 13)),
    Conv((3, 3), 6 => 16, relu),
    AdaptiveMaxPool((5, 5)),
    flatten,
    Dense(400, 100, relu),
    Dense(100, 84, relu),
    Dense(84, 10),
    softmax
)

In [ ]:
# Train the model
loss = Flux.Losses.crossentropy
opt  = Flux.Optimise.ADAM()

train_iterator = DataLoader(traindata, batchsize=250, shuffle=true)
valid_iterator = DataLoader(valdata,   batchsize=250, shuffle=true)
learner        = Learner(model, (train_iterator, valid_iterator),
                         opt, loss, Metrics(accuracy), ToGPU())

nepochs = 10
FluxTraining.fit!(learner, nepochs)

In [ ]:
# Plot learning rate
training_loss   = learner.cbstate.metricsepoch[TrainingPhase()][:Loss].values
validation_loss = learner.cbstate.metricsepoch[ValidationPhase()][:Loss].values

p = plot(training_loss, label="Training loss")
p = plot!(p, validation_loss, label="Validation loss")

In [ ]:
# Evaluate the model.
test_pred = model(testdata[1])

@show loss(test_pred, testdata[2])
@show accuracy(test_pred, testdata[2]);

In [ ]:
plots = map(1:25) do i
    p = plot(Gray.(1 .- testdata[1][:, :, 1, i])'; yaxis=nothing, xaxis=nothing)
    prediction = findmax(vec(model(testdata[1][:, :, :, i:i])))
    title!(p, "$(prediction[2]-1) -> $(round(100prediction[1], digits=1))%",
    titlefontsize=8)
end
plot(plots...)

# Deep convolutional autoencoder

![](auto.png)

In [ ]:
# Build the autoencoder network.
encoder = Chain(
    Conv((3, 3),  1 => 16, relu, pad=SamePad()),
    MaxPool((2, 2),              pad=SamePad()),
    Conv((3, 3), 16 =>  8, relu, pad=SamePad()),
    MaxPool((2, 2),              pad=SamePad()),
    Conv((3, 3),  8 =>  2, relu, pad=SamePad()),
    MaxPool((2, 2),              pad=SamePad()),
)

decoder = Chain(
    Conv((3, 3),  2 =>  2, relu, pad=SamePad()),
    Upsample(scale=(2, 2)),
    Conv((3, 3),  2 =>  8, relu, pad=SamePad()),
    Upsample(scale=(2, 2)),
    Conv((3, 3),  8 => 16, relu),
    Upsample(scale=(2, 2)),
    Conv((3, 3), 16 => 1,  sigmoid, pad=SamePad()),
)
    
autoencoder = Chain(encoder, decoder)

In [ ]:
# Gather training data for autoencoder:
traindata, valdata, testdata = splitobs((x_data, x_data), at=(0.9, 0.05))

# Train the model
loss = Flux.Losses.mse
opt  = Flux.Optimise.ADADelta()

train_iterator = DataLoader(traindata, batchsize=128, shuffle=true)
valid_iterator = DataLoader(valdata,   batchsize=128, shuffle=true)
learner        = Learner(autoencoder, (train_iterator, valid_iterator),
                         opt, loss, ToGPU())

nepochs = 30
FluxTraining.fit!(learner, nepochs)

In [ ]:
# Plot learning rate
training_loss   = learner.cbstate.metricsepoch[TrainingPhase()][:Loss].values
validation_loss = learner.cbstate.metricsepoch[ValidationPhase()][:Loss].values

p = plot(training_loss, label="Training loss")
p = plot!(p, validation_loss, label="Validation loss")

In [ ]:
# Evaluate the model.
@show loss(autoencoder(testdata[1]), testdata[2])

In [ ]:
# Encode and then decode some test data.
encoded_imgs = encoder(testdata[1])
decoded_imgs = decoder(encoded_imgs);

In [ ]:
plots = map(1:10) do i
    p = plot(Gray.(1 .- testdata[1][:, :, 1, i])';  yaxis=nothing, xaxis=nothing)
    q = plot(Gray.(1 .- decoded_imgs[:, :, 1, i])'; yaxis=nothing, xaxis=nothing)
    (p, q)
end
plot(plots..., layout=(10, 2))

In [ ]:
# Plot the latent spaces.
plots = map(1:10) do i
    latent_reshaped = reshape(encoded_imgs[:, :, :, i], 4, 8)
    plot(Gray.(1 .- latent_reshaped); yaxis=nothing, xaxis=nothing)
end
plot(plots...)

## Solving the Schrödinger Equation using Deep Learning

In [ ]:
# Use pickle to import data and get it to the julia world
py"""
import pickle
with open('V.db', 'rb') as f:
    V = pickle.load(f)
with open('density.db', 'rb') as f:
    n = pickle.load(f)
with open('E.db', 'rb') as f:
    E = pickle.load(f)
"""
V, ρ, E = py"(V, n, E)"
n_samples, n_grid = size(V)

# Reshape appropriately
V = reshape(Float32.(V'), n_grid, 1, n_samples)
ρ = reshape(Float32.(ρ'), n_grid, 1, n_samples)
E = reshape(Float32.(E),          1, n_samples);

# Split into training, validation, testing
traindata, valdata, testdata = splitobs((V, E), at=(0.5, 0.25))

# Show sizes
@show size(traindata[1]) size(valdata[1]) size(testdata[1])
@show size(traindata[2]) size(valdata[2]) size(testdata[2]);

In [ ]:
# Build the neural network.
V_to_E = Chain(
    Conv((3, ), 1 => 6, relu),
    MaxPool((2, )),
    Conv((3, ), 6 => 16, relu),
    MaxPool((2, )),
    flatten,
    Dense(224, 120, relu),
    # Dropout(0.5),
    Dense(120, 50, relu),
    # Dropout(0.5),
    Dense(50, 1),
)

In [ ]:
# Train the model
loss = Flux.Losses.mse
opt  = Flux.Optimise.ADADelta()

train_iterator = DataLoader(traindata, batchsize=250, shuffle=true)
valid_iterator = DataLoader(valdata,   batchsize=250, shuffle=true)
learner        = Learner(V_to_E, (train_iterator, valid_iterator),
                         opt, loss, ToGPU())

nepochs = 20
FluxTraining.fit!(learner, nepochs)

In [ ]:
# Plot learning rate
training_loss   = learner.cbstate.metricsepoch[TrainingPhase()][:Loss].values
validation_loss = learner.cbstate.metricsepoch[ValidationPhase()][:Loss].values

p = plot(training_loss, label="Training loss")
p = plot!(p, validation_loss, label="Validation loss")

In [ ]:
# Evaluate the model.
@show loss(V_to_E(testdata[1]), testdata[2])

In [ ]:
E_pred = V_to_E(testdata[1])
plots = map(enumerate(CartesianIndices((5, 5)))) do (k, I)
    # ρ_test[:, 1, k]
    p = plot(testdata[1][:, 1, k]; yaxis=nothing, xaxis=nothing, color=:red, label="")
    hline!(p, testdata[2][:, k]; color=:blue, linestyle=:dash, label="")
    hline!(p, E_pred[:, k]; color=:green, label="")
    ylims!(p, (NaN, 0.5))
end
plot(plots...)